In [1]:
import pandas as pd
import os

SEQ_LEN=60
FUTURE_PERIOD_PREDICT=3
RATIO_TO_PREDICT="LTC-USD"

def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0
    
main_df = pd.DataFrame()

ratios= ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/crypto_data/{ratio}.csv"
    df=pd.read_csv(dataset, names=["time", "low", "high", "open", "close", "volume"])
    #inplace True so that we don't have to redefine df
    df.rename(columns={"close":f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace=True)
    df.set_index("time", inplace=True)
    
    df=df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    if len(main_df)==0:
        main_df=df
    else:
        main_df=main_df.join(df)
        
main_df['future']=main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)

main_df['target']=list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

print(main_df[[f"{RATIO_TO_PREDICT}_close", "future", "target"]].head(10))

            LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
1528969020      96.440002  96.400002       0
1528969080      96.470001  96.400002       0
1528969140      96.400002  96.400002       0
1528969200      96.400002  96.400002       0


In [2]:
from sklearn import preprocessing
from collections import deque
from random import shuffle
import numpy as np

In [3]:
def preprocess_df(df):
    df=df.drop('future',1)
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()#normalize
            df.dropna(inplace=True)
            df[col]=preprocessing.scale(df[col].values)#scale

    df.dropna(inplace=True)

    sequential_data=[]
    prev_days=deque(maxlen=SEQ_LEN)

    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days)==SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]])
            shuffle(sequential_data)
            
    buys, sells = [], []
    for seq, target in sequential_data:
        if target==0:
            sells.append([seq, target])
        elif target==1:
            buys.append([seq, target])
            
    shuffle(buys);shuffle(sells)
    
    lower = min(len(buys), len(sells))
    buys, sells = buys[:lower], sells[:lower]
    
    sequential_data = buys+sells
    
    shuffle(sequential_data)
    
    X, y = [], []
    
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
        
    return np.array(X), np.array(y)

In [ ]:
times=sorted(main_df.index.values)
last_5pct=times[-int(0.05*len(times))]

validation_main_df=main_df[(main_df.index >= last_5pct)]
main_df=main_df[(main_df.index < last_5pct)]

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

print(f"train_data: {len(train-x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_train_y.count(0)}, VALIDATION buys: {validation_train_y.count(1)}")